#  TP2 - Clustering (Sans Réduction de Dimension)

 Ce notebook charge les données Hi-Seq normalisées (`hiseq_X_scaled.npy`) et applique les algorithmes K-Means, DBSCAN et Clustering Spectral sur l'ensemble des attributs.

In [24]:
# --- Imports et GESTION DES CHEMINS ---
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import warnings
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import time
import tracemalloc
import warnings
import sys
import os
from pathlib import Path

In [25]:

# Chemin actuel: notebooks/clustering -> Remonter de 2 niveaux
# --- Configuration des Chemins ---

PROJECT_ROOT = Path("../..").resolve()
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "HISEQ"
PROJECT_FCT = PROJECT_ROOT / "src"
sys.path.append(str(PROJECT_FCT))

print(f"Chemin du projet racine : {PROJECT_ROOT}")
print(f"Chargement des données depuis : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Chargement des données depuis : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data\HISEQ


In [26]:
# --- Imports depuis vos fichiers utilitaires ---
try:
    from utils import (
        evaluate_clustering,
        run_stochastic_protocol,
        print_protocol_summary,
        run_deterministic_protocol,
        plot_clusters 
    )
    from preprocess import (
        reduce_dimension_for_clustering,
        reduce_dimension_for_viz  
    )
except ImportError:
    print("ERREUR: Impossible d'importer les fonctions utilitaires depuis 'tp2_utils' ou 'tp2_preprocess'.")
    print("Vérifiez que la racine du projet est correcte et que les fichiers .py s'y trouvent.")

warnings.filterwarnings('ignore')

In [27]:
# --- Chargement des données ---
print("Chargement des données Hi-Seq normalisées...")
try:
    X_scaled = np.load(DATA_PROCESSED_PATH / 'hiseq_X_scaled.npy')
    y_hiseq = pd.read_csv(DATA_PROCESSED_PATH / 'hiseq_y_labels.csv', index_col=0).iloc[:, 0]
    print(f"Données chargées. Forme de X_scaled: {X_scaled.shape}")
except FileNotFoundError:
    print(f"ERREUR: Fichiers non trouvés dans {DATA_PROCESSED_PATH}")
    print("Veuillez d'abord exécuter le notebook 'PreparationData.ipynb'.")

Chargement des données Hi-Seq normalisées...
Données chargées. Forme de X_scaled: (801, 20531)


In [28]:
# Créer un DataFrame pour les fonctions de visualisation (elles attendent un DF)
# L'index doit correspondre à y_hiseq
X_scaled_df = pd.DataFrame(X_scaled, index=y_hiseq.index)

# --- Constantes pour le protocole ---
N_RUNS = 10
N_CLUSTERS = 5 # 5 classes dans HiSeq
FIXED_SEED = 42

# 1. Clustering sur Données Complètes

1.1 K-Means (Full) - Protocole expérimental (10 exécutions)

In [29]:
# Définir la "fabrique" (factory) pour KMeans
kmeans_factory = lambda seed: KMeans(n_clusters=N_CLUSTERS,
                                     random_state=seed,
                                     n_init=10)

In [30]:
# Exécuter le protocole sur les données complètes X_scaled
df_kmeans_full = run_stochastic_protocol(
    model_factory=kmeans_factory,
    data=X_scaled,
    true_labels=y_hiseq,
    n_runs=N_RUNS,
    experiment_name="KMeans (Full)",
    evaluate_clustering_func=evaluate_clustering
)

--- Protocole expérimental : KMeans (Full) ---
  Exécution 1/10 (KMeans (Full))...
  Exécution 2/10 (KMeans (Full))...
  Exécution 3/10 (KMeans (Full))...
  Exécution 4/10 (KMeans (Full))...
  Exécution 5/10 (KMeans (Full))...
  Exécution 6/10 (KMeans (Full))...
  Exécution 7/10 (KMeans (Full))...
  Exécution 8/10 (KMeans (Full))...
  Exécution 9/10 (KMeans (Full))...
  Exécution 10/10 (KMeans (Full))...
Protocole KMeans (Full) terminé.


In [31]:
# Afficher le résumé
results_kmeans_full_mean = print_protocol_summary(df_kmeans_full, "KMeans (Full)")


--- Résultats KMeans (Full) - (μ ± σ) sur 10 exécutions ---
Silhouette:        0.1353 ± 0.0003
Homogeneity:       0.8644 ± 0.0025
Completeness:      0.8504 ± 0.0025
V-Measure:         0.8573 ± 0.0025
ARI:               0.8008 ± 0.0056
Time (s):          1.9641 ± 0.1555
Memory (MB):       251.4550 ± 0.0002


1.2 DBSCAN (Full)

In [33]:
print("Calcul de EPS pour DBSCAN (Full)...")
neighbors_full = NearestNeighbors(n_neighbors=5).fit(X_scaled)
distances_full, _ = neighbors_full.kneighbors(X_scaled)
distances_full = np.sort(distances_full, axis=0)[:, 4]
kneedle_full = KneeLocator(range(len(distances_full)), distances_full, curve='convex', direction='increasing')
# Utilisation d'une valeur de repli (fallback) trouvée précédemment si kneedle échoue
EPS_FULL_OPTIMAL = kneedle_full.elbow_y if kneedle_full.elbow_y else 170.0
print(f"EPS Optimal (Full) trouvé: {EPS_FULL_OPTIMAL}")

Calcul de EPS pour DBSCAN (Full)...
EPS Optimal (Full) trouvé: 172.66466987135783


In [34]:
# Initialiser le modèle
dbscan_full_model = DBSCAN(eps=EPS_FULL_OPTIMAL, min_samples=5)

# Exécuter le protocole déterministe
results_dbscan_full_series = run_deterministic_protocol(
    model=dbscan_full_model,
    data=X_scaled,
    true_labels=y_hiseq,
    experiment_name="DBSCAN (Full)",
    evaluate_clustering_func=evaluate_clustering
)

--- Exécution : DBSCAN (Full) ---
Exécution DBSCAN (Full) terminée.

--- Résultats DBSCAN (Full) - (1 exécution) ---
|                  |        0 |
|:-----------------|---------:|
| Silhouette       | nan      |
| Homogeneity      |   0.0000 |
| Completeness     |   1.0000 |
| V-Measure        |   0.0000 |
| ARI              |   0.0000 |
| Noise_Percentage |   5.7428 |
| Time (s)         |   0.8546 |
| Memory (MB)      | 357.7744 |


1.3 Clustering Spectral (Données Complètes)

In [35]:
print("--- Protocole expérimental : Spectral Clustering (Full) ---")

results_spectral_full_series = pd.Series(dtype=float)

# Définir la "fabrique" pour Spectral Clustering
# n_init=1 pour le k-means interne, crucial pour la vitesse !
spectral_factory = lambda seed: SpectralClustering(n_clusters=N_CLUSTERS,assign_labels='kmeans',random_state=seed,n_init=1) 

# Exécuter le protocole complet
df_spectral_full = run_stochastic_protocol(
    model_factory=spectral_factory,
    data=X_scaled,
    true_labels=y_hiseq,
    n_runs=N_RUNS, # N_RUNS = 10
    experiment_name="Spectral (Full)",
    evaluate_clustering_func=evaluate_clustering
)

# Afficher le résumé si le protocole a réussi
results_spectral_full_mean = print_protocol_summary(df_spectral_full, "Spectral (Full)")



--- Protocole expérimental : Spectral Clustering (Full) ---
--- Protocole expérimental : Spectral (Full) ---
  Exécution 1/10 (Spectral (Full))...
  Exécution 2/10 (Spectral (Full))...
  Exécution 3/10 (Spectral (Full))...
  Exécution 4/10 (Spectral (Full))...
  Exécution 5/10 (Spectral (Full))...
  Exécution 6/10 (Spectral (Full))...
  Exécution 7/10 (Spectral (Full))...
  Exécution 8/10 (Spectral (Full))...
  Exécution 9/10 (Spectral (Full))...
  Exécution 10/10 (Spectral (Full))...
Protocole Spectral (Full) terminé.

--- Résultats Spectral (Full) - (μ ± σ) sur 10 exécutions ---
Silhouette:        -0.0376 ± 0.0261
Homogeneity:       0.0065 ± 0.0033
Completeness:      0.0173 ± 0.0088
V-Measure:         0.0093 ± 0.0046
ARI:               -0.0024 ± 0.0043
Time (s):          0.4284 ± 0.2963
Memory (MB):       20.9512 ± 0.0002


3. Comparaison Finale (Données Complètes)

In [36]:
data_summary = {
    ('Full', 'KMeans'): results_kmeans_full_mean,
    ('Full', 'DBSCAN'): results_dbscan_full_series,
    ('Full', 'Spectral'): results_spectral_full_mean
}

# %% [code]
df_final_results = pd.DataFrame(data_summary).T
df_final_results.index.names = ['Jeu', 'Algo']

# Remplacer les NaT/NaN (provenant d'une exécution Spectral échouée) par 0 pour l'affichage
df_final_results = df_final_results.fillna(0)

print("\n--- Tableau Récapitulatif (Moyennes des 10 exécutions) ---")
# Assurer l'ordre des colonnes tel que défini dans vos utils
columns_order = ['ARI', 'V-Measure', 'Silhouette', 'Noise_Percentage', 'Time (s)', 'Memory (MB)', 'Homogeneity', 'Completeness']
final_columns = [col for col in columns_order if col in df_final_results.columns]
print(df_final_results[final_columns].to_markdown(floatfmt=".4f"))


--- Tableau Récapitulatif (Moyennes des 10 exécutions) ---
|                      |     ARI |   V-Measure |   Silhouette |   Noise_Percentage |   Time (s) |   Memory (MB) |   Homogeneity |   Completeness |
|:---------------------|--------:|------------:|-------------:|-------------------:|-----------:|--------------:|--------------:|---------------:|
| ('Full', 'KMeans')   |  0.8008 |      0.8573 |       0.1353 |             0.0000 |     1.9641 |      251.4550 |        0.8644 |         0.8504 |
| ('Full', 'DBSCAN')   |  0.0000 |      0.0000 |       0.0000 |             5.7428 |     0.8546 |      357.7744 |        0.0000 |         1.0000 |
| ('Full', 'Spectral') | -0.0024 |      0.0093 |      -0.0376 |             0.0000 |     0.4284 |       20.9512 |        0.0065 |         0.0173 |
